In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My\ Drive/PRML-2020/Data\ Contest

/content/drive/My Drive/PRML-2020/Data Contest


In [3]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import pickle as pkl

# Train set preparation

In [4]:
train_df = pd.read_csv('./dataset/train.csv')
time = []
format = '%d-%m-%Y %H:%M:%S'
for t in tqdm(train_df['timestamp']):
    dt_obj = datetime.strptime(t, format)
    time.append(86400*(dt_obj.year*1e6+dt_obj.month*1e2+dt_obj.day)+3600*dt_obj.hour+60*dt_obj.minute+dt_obj.second)
    
train_df['timestamp'] = time

biker_ids = train_df['biker_id'].unique()
tour_ids = train_df['tour_id'].unique()
print("No. of unique bikers : {0}".format(len(biker_ids)))
print("No. of unique tours : {0}".format(len(tour_ids)))
display(train_df)


No. of unique bikers : 1934
No. of unique tours : 7985


,biker_id,tour_id,invited,timestamp,like,dislike
0,DA44012,QY18771225,0,1.738369e+14,0,0
1,DA44012,QU02284248,0,1.738369e+14,0,0
2,DA44012,RU29072432,0,1.738369e+14,1,0
3,DA44012,SP72478280,0,1.738369e+14,0,0
4,DA44012,QS90707377,0,1.738369e+14,0,0
...,...,...,...,...,...,...
13861,EC93103086,RW50873665,0,1.738369e+14,0,0
13862,EC93103086,TP84655790,0,1.738369e+14,0,0
13863,EC93103086,UY8708806,0,1.738369e+14,0,0
13864,EC93103086,VP4179853,0,1.738369e+14,0,0


In [5]:
tours_info = pd.read_csv('./Required data/tours_info.csv')

train_df_full = train_df.merge(right=tours_info, how='left', on='tour_id')
display(train_df_full)

,biker_id,tour_id,invited,timestamp,like,dislike,tour_date,latitude,longitude,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20,w21,w22,w23,w24,w25,w26,w27,w28,w29,w30,w31,...,w63,w64,w65,w66,w67,w68,w69,w70,w71,w72,w73,w74,w75,w76,w77,w78,w79,w80,w81,w82,w83,w84,w85,w86,w87,w88,w89,w90,w91,w92,w93,w94,w95,w96,w97,w98,w99,w100,w_other,host_id
0,DA44012,QY18771225,0,1.738369e+14,0,0,20121003,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,38209
1,DA44012,QU02284248,0,1.738369e+14,0,0,20121003,-7.767,110.363,2,0,0,0,2,3,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,38209
2,DA44012,RU29072432,0,1.738369e+14,1,0,20121026,3.567,98.650,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,37,38209
3,DA44012,SP72478280,0,1.738369e+14,0,0,20121006,34.017,71.583,1,2,1,1,0,0,2,0,0,1,1,1,2,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0,56,38209
4,DA44012,QS90707377,0,1.738369e+14,0,0,20121006,3.607,98.653,2,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,79,38209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13861,EC93103086,RW50873665,0,1.738369e+14,0,0,20121208,-6.973,107.633,1,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,38209
13862,EC93103086,TP84655790,0,1.738369e+14,0,0,20121209,0.000,0.000,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,38209
13863,EC93103086,UY8708806,0,1.738369e+14,0,0,20121209,-7.973,112.609,3,2,0,1,1,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,60,38209
13864,EC93103086,VP4179853,0,1.738369e+14,0,0,20121212,-8.167,113.700,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,38209


In [6]:
bikers_info = pd.read_csv('./Required data/bikers_info.csv')
bikers_info.rename(columns={'Unnamed: 0':'biker_id'}, inplace=True)

train_df_full = train_df_full.merge(right=bikers_info, how='left', on='biker_id')
display(train_df_full)

,biker_id,tour_id,invited,timestamp,like,dislike,tour_date,latitude,longitude,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20,w21,w22,w23,w24,w25,w26,w27,w28,w29,w30,w31,...,area_Woodland CA,area_Woodland Hills CA,area_Woodland Hills California,area_Woodlands Singapore,area_Woodside California,area_Woodstock ON,area_Woodstock Ontario,area_Worcester Q4,area_Wuhan China,area_Xiamen Shi Fujian China,area_Yangon Burma,area_Yantai 25,area_Yazd Iran,area_Yekaterinburg 71,area_Yerevan 11,area_Yerevan Armenia,area_Yio Chu Kang Singapore,area_Yogyakarta,area_Yogyakarta 10,area_Yokohama 19,area_Yokosuka-shi Kanagawa Japan,area_Yola Nigeria,area_Yonkers New York,area_Yorba Linda CA,area_York ON,area_Zabol,area_Zagazig 14,area_Zagreb Croatia,area_Zamboanga G7,area_Zamboanga City,area_Zanjan Iran,area_Zaragoza Spain,area_Zaria Nigeria,area_Zarqa' Az Zarqa Jordan,area_Zhengzhou,area_Zhonghe Heilongjiang China,area_Zonhoven,area_Zurich 25,area_`Arab Jizan Saudi Arabia,area_undefined undefined
0,DA44012,QY18771225,0,1.738369e+14,0,0,20121003,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,DA44012,QU02284248,0,1.738369e+14,0,0,20121003,-7.767,110.363,2,0,0,0,2,3,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,DA44012,RU29072432,0,1.738369e+14,1,0,20121026,3.567,98.650,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,DA44012,SP72478280,0,1.738369e+14,0,0,20121006,34.017,71.583,1,2,1,1,0,0,2,0,0,1,1,1,2,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,DA44012,QS90707377,0,1.738369e+14,0,0,20121006,3.607,98.653,2,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13861,EC93103086,RW50873665,0,1.738369e+14,0,0,20121208,-6.973,107.633,1,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13862,EC93103086,TP84655790,0,1.738369e+14,0,0,20121209,0.000,0.000,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13863,EC93103086,UY8708806,0,1.738369e+14,0,0,20121209,-7.973,112.609,3,2,0,1,1,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13864,EC93103086,VP4179853,0,1.738369e+14,0,0,20121212,-8.167,113.700,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
bikers_network = pkl.load(open('./Required data/train_biker_network.pkl', 'rb'))
tour_convoy = pkl.load(open('./Required data/train_tour_convoy.pkl', 'rb'))

In [8]:
num_friends_going = []
num_friends_not_going = []
num_friends_maybe = []
num_friends_invited = []
for index in tqdm(train_df_full.index):
    biker_id = train_df_full.iloc[index]['biker_id']
    tour_id = train_df_full.iloc[index]['tour_id']
    friends = bikers_network[biker_id]
    tour_going = tour_convoy[tour_id]['going']
    tour_not_going = tour_convoy[tour_id]['not_going']
    tour_maybe = tour_convoy[tour_id]['maybe']
    tour_invited = tour_convoy[tour_id]['invited']
    
    num_frnd_going, num_frnd_not_going, num_frnd_maybe, num_frnd_invited = 0, 0, 0, 0
    for frnd in friends:
        if frnd in tour_going:
            num_frnd_going += 1
        elif frnd in tour_not_going:
            num_frnd_not_going += 1
        elif frnd in tour_maybe:
            num_frnd_maybe += 1
        elif frnd in tour_invited:
            num_frnd_invited += 1
    
    num_friends_going.append(num_frnd_going)
    num_friends_not_going.append(num_frnd_not_going)
    num_friends_maybe.append(num_frnd_maybe)
    num_friends_invited.append(num_frnd_invited)

In [9]:
train_df_full['num_friends_going'] = num_friends_going
train_df_full['num_friends_not_going'] = num_friends_not_going
train_df_full['num_friends_maybe'] = num_friends_maybe
train_df_full['num_friends_invited'] = num_friends_invited

display(train_df_full)

,biker_id,tour_id,invited,timestamp,like,dislike,tour_date,latitude,longitude,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20,w21,w22,w23,w24,w25,w26,w27,w28,w29,w30,w31,...,area_Woodside California,area_Woodstock ON,area_Woodstock Ontario,area_Worcester Q4,area_Wuhan China,area_Xiamen Shi Fujian China,area_Yangon Burma,area_Yantai 25,area_Yazd Iran,area_Yekaterinburg 71,area_Yerevan 11,area_Yerevan Armenia,area_Yio Chu Kang Singapore,area_Yogyakarta,area_Yogyakarta 10,area_Yokohama 19,area_Yokosuka-shi Kanagawa Japan,area_Yola Nigeria,area_Yonkers New York,area_Yorba Linda CA,area_York ON,area_Zabol,area_Zagazig 14,area_Zagreb Croatia,area_Zamboanga G7,area_Zamboanga City,area_Zanjan Iran,area_Zaragoza Spain,area_Zaria Nigeria,area_Zarqa' Az Zarqa Jordan,area_Zhengzhou,area_Zhonghe Heilongjiang China,area_Zonhoven,area_Zurich 25,area_`Arab Jizan Saudi Arabia,area_undefined undefined,num_friends_going,num_friends_not_going,num_friends_maybe,num_friends_invited
0,DA44012,QY18771225,0,1.738369e+14,0,0,20121003,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3
1,DA44012,QU02284248,0,1.738369e+14,0,0,20121003,-7.767,110.363,2,0,0,0,2,3,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,DA44012,RU29072432,0,1.738369e+14,1,0,20121026,3.567,98.650,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,DA44012,SP72478280,0,1.738369e+14,0,0,20121006,34.017,71.583,1,2,1,1,0,0,2,0,0,1,1,1,2,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,DA44012,QS90707377,0,1.738369e+14,0,0,20121006,3.607,98.653,2,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13861,EC93103086,RW50873665,0,1.738369e+14,0,0,20121208,-6.973,107.633,1,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
13862,EC93103086,TP84655790,0,1.738369e+14,0,0,20121209,0.000,0.000,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
13863,EC93103086,UY8708806,0,1.738369e+14,0,0,20121209,-7.973,112.609,3,2,0,1,1,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,3
13864,EC93103086,VP4179853,0,1.738369e+14,0,0,20121212,-8.167,113.700,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [10]:
array = train_df_full.to_numpy()[:, 2:].astype(np.float)
columns = train_df_full.columns
rm_cols = columns[np.where(np.sum(array, axis=0)==0)][1:]

train_df_full.drop(rm_cols, axis=1, inplace=True)
display(train_df_full)
train_df_full.to_csv('./Required data/train_complete.csv')

,biker_id,tour_id,invited,timestamp,like,dislike,tour_date,latitude,longitude,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20,w21,w22,w23,w24,w25,w26,w27,w28,w29,w30,w31,...,area_Tel Aviv Israel,area_Temecula California,area_Termo De La Spezia Liguria Italy,area_The Plains Ohio,area_Toluca 15,area_Torino 12,area_Toronto ON,area_Toronto Ohio,area_Toyama-shi Toyama Japan,area_Tracy California,area_Tripoli 61,area_Tripoli Libya,area_Tuban Jawa Timur Indonesia,area_Turbenthal Switzerland,area_Ubon Rachatani Ubon Ratchathani Thailand,area_Union City CA,area_Varanassi Uttar Pradesh India,area_Varginha Brazil,area_Vieux Grand Port Grand Port Mauritius,area_Villa De Laguna Salada Valverde Dominican Republic,area_Villa Mella Dominican Republic,area_Vung Tau 45,area_Washington Heights New York,area_Weifang,area_Westchester New York,area_Whitby Ontario,area_Wolcott Connecticut,area_Wonogiri Jawa Tengah Indonesia,area_Woodland CA,area_Yerevan Armenia,area_Yio Chu Kang Singapore,area_Yokosuka-shi Kanagawa Japan,area_Zarqa' Az Zarqa Jordan,area_Zurich 25,area_`Arab Jizan Saudi Arabia,area_undefined undefined,num_friends_going,num_friends_not_going,num_friends_maybe,num_friends_invited
0,DA44012,QY18771225,0,1.738369e+14,0,0,20121003,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3
1,DA44012,QU02284248,0,1.738369e+14,0,0,20121003,-7.767,110.363,2,0,0,0,2,3,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,DA44012,RU29072432,0,1.738369e+14,1,0,20121026,3.567,98.650,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,DA44012,SP72478280,0,1.738369e+14,0,0,20121006,34.017,71.583,1,2,1,1,0,0,2,0,0,1,1,1,2,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,DA44012,QS90707377,0,1.738369e+14,0,0,20121006,3.607,98.653,2,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13861,EC93103086,RW50873665,0,1.738369e+14,0,0,20121208,-6.973,107.633,1,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
13862,EC93103086,TP84655790,0,1.738369e+14,0,0,20121209,0.000,0.000,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
13863,EC93103086,UY8708806,0,1.738369e+14,0,0,20121209,-7.973,112.609,3,2,0,1,1,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,3
13864,EC93103086,VP4179853,0,1.738369e+14,0,0,20121212,-8.167,113.700,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
